In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

In [2]:
# Set up the Selenium WebDriver to connect to the Selenium server (replace 'selenium' with your service name if different)
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode to avoid opening a UI
driver = webdriver.Remote(
    command_executor='http://selenium:4444/wd/hub',
    options=options
)

In [3]:
# Go to the Letterboxd popular movies page
driver.get("https://letterboxd.com/films/popular/page/1/")

In [5]:
# Find all elements that contain movie links
movie_elements = driver.find_elements(By.CSS_SELECTOR, 'a.frame')

# Extract URLs from the elements
movie_urls = [element.get_attribute('href') for element in movie_elements]

In [6]:
movie_urls

['https://letterboxd.com/film/barbie/',
 'https://letterboxd.com/film/parasite-2019/',
 'https://letterboxd.com/film/everything-everywhere-all-at-once/',
 'https://letterboxd.com/film/fight-club/',
 'https://letterboxd.com/film/la-la-land/',
 'https://letterboxd.com/film/interstellar/',
 'https://letterboxd.com/film/oppenheimer-2023/',
 'https://letterboxd.com/film/joker-2019/',
 'https://letterboxd.com/film/dune-2021/',
 'https://letterboxd.com/film/pulp-fiction/',
 'https://letterboxd.com/film/whiplash-2014/',
 'https://letterboxd.com/film/spider-man-into-the-spider-verse/',
 'https://letterboxd.com/film/get-out-2017/',
 'https://letterboxd.com/film/the-batman/',
 'https://letterboxd.com/film/the-dark-knight/',
 'https://letterboxd.com/film/saltburn/',
 'https://letterboxd.com/film/the-truman-show/',
 'https://letterboxd.com/film/spider-man-across-the-spider-verse/',
 'https://letterboxd.com/film/lady-bird/',
 'https://letterboxd.com/film/american-psycho/',
 'https://letterboxd.com/f

In [1]:
import argparse
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

def scrape_movie_urls(num_movies):
    # Setup WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Remote(
        command_executor='http://selenium:4444/wd/hub',
        options=options
    )
    movie_urls = []
    page = 1
    while len(movie_urls) < num_movies:
        driver.get(f"https://letterboxd.com/films/popular/page/{page}/")
        elements = driver.find_elements(By.CSS_SELECTOR, 'a.frame')
        for element in elements:
            if len(movie_urls) >= num_movies:
                break
            movie_urls.append(element.get_attribute('href'))
        page += 1
    
    df = pd.DataFrame(movie_urls, columns=['url'])
    return df
    #df.to_csv('/app/data/movie_urls.csv', index=False)


In [2]:
x = scrape_movie_urls(5)
x

,url
0,https://letterboxd.com/film/barbie/
1,https://letterboxd.com/film/parasite-2019/
2,https://letterboxd.com/film/everything-everywh...
3,https://letterboxd.com/film/fight-club/
4,https://letterboxd.com/film/la-la-land/


In [3]:
def normalize_rating(star_rating):
    rating_map = {'★': 0.2, '½': 0.1}
    return sum(rating_map[char] for char in star_rating if char in rating_map)

def scrape_reviews_per_movie(num_reviews_per_movie):
    # Setup WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    print("Attempting to connect to Selenium WebDriver...")
    
    driver = webdriver.Remote(
        command_executor='http://selenium:4444/wd/hub',
        options=options
    )
    print("Successfully connected to WebDriver!")
    
    movie_urls = pd.read_csv('/app/data/movie_urls.csv')['url'].tolist()
    all_reviews = []

    for movie_url in movie_urls:
        print(f"Scraping reviews for movie: {movie_url}")
        movie_reviews = []
        driver.get(f"{movie_url}/reviews/")
        
        page = 1
        while len(movie_reviews) < num_reviews_per_movie:
            review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.film-detail-content')
            
            for element in review_elements:
                if len(movie_reviews) >= num_reviews_per_movie:
                    break
                
                review_text_element = element.find_element(By.CSS_SELECTOR, 'div.body-text.-prose.collapsible-text')
                review_text = review_text_element.text if review_text_element else None
                
                rating_element = element.find_element(By.CSS_SELECTOR, 'span.rating')
                star_rating = rating_element.text if rating_element else None
                
                if review_text and star_rating is not None:
                    movie_reviews.append({
                        'movie_url': movie_url,
                        'review': review_text,
                        'rating': normalize_rating(star_rating)
                    })
            
            # Go to next page if needed
            if len(movie_reviews) < num_reviews_per_movie:
                page += 1
                driver.get(f"{movie_url}/reviews/page/{page}/")
            else:
                break
        
        all_reviews.extend(movie_reviews)
    
    df = pd.DataFrame(all_reviews)
    return df

    #df.to_csv('/app/data/movie_reviews.csv', index=False)

In [12]:
y = scrape_movie_urls(1)

In [13]:
y

,url
0,https://letterboxd.com/film/barbie/


# Organize code a bit more

In [1]:
import argparse
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

## 1a) Scrape movie urls

In [2]:
def scrape_movie_urls(num_movies):
    # Setup WebDriver
    print("Starting to scrape movie urls!\n")
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    print("Connecting to selenium service....")
    driver = webdriver.Remote(
        command_executor='http://selenium:4444/wd/hub',
        options=options
    )
    print("...Connected to selenium service!")
    movie_urls = []
    page = 1
    while len(movie_urls) < num_movies:
        driver.get(f"https://letterboxd.com/films/popular/page/{page}/")
        elements = driver.find_elements(By.CSS_SELECTOR, 'a.frame')
        for element in elements:
            if len(movie_urls) >= num_movies:
                break
            movie_urls.append(element.get_attribute('href'))
        page += 1
    
    df = pd.DataFrame(movie_urls, columns=['url'])
    print("Movie urls sucessfully scraped!\nMovie urls preview:")
    print(df.head().to_string())
    print("\nClosing connection to selenium webdriver...")
    driver.quit()
    print("Connection closed sucessfully!\n\n")
    return df

In [3]:
movie_urls = scrape_movie_urls(5)
movie_urls

Starting to scrape movie urls!

Connecting to selenium service....
...Connected to selenium service!
Movie urls sucessfully scraped!
Movie urls preview:
                                                              url
0                             https://letterboxd.com/film/barbie/
1                      https://letterboxd.com/film/parasite-2019/
2  https://letterboxd.com/film/everything-everywhere-all-at-once/
3                         https://letterboxd.com/film/fight-club/
4                         https://letterboxd.com/film/la-la-land/

Closing connection to selenium webdriver...
Connection closed sucessfully!




,url
0,https://letterboxd.com/film/barbie/
1,https://letterboxd.com/film/parasite-2019/
2,https://letterboxd.com/film/everything-everywh...
3,https://letterboxd.com/film/fight-club/
4,https://letterboxd.com/film/la-la-land/


In [8]:
for movie_url in movie_urls.url:
    print(movie_url)

https://letterboxd.com/film/barbie/
https://letterboxd.com/film/parasite-2019/
https://letterboxd.com/film/everything-everywhere-all-at-once/
https://letterboxd.com/film/fight-club/
https://letterboxd.com/film/la-la-land/


## 1b) Scrape reviews from movie urls

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, TimeoutException

def normalize_rating(star_rating):
    rating_map = {'★': 0.2, '½': 0.1}
    return sum(rating_map[char] for char in star_rating if char in rating_map)


def scrape_reviews_from_movies(movie_urls, num_reviews_per_movie):
    # Setup WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    print("Attempting to connect to Selenium WebDriver...")
    driver = webdriver.Remote(
        command_executor='http://selenium:4444/wd/hub',
        options=options
    )
    print("Successfully connected to WebDriver!")
    
    all_reviews = []

    for movie_url in movie_urls.url:
        print(f"Scraping reviews for movie: {movie_url}")
        movie_reviews = []
        page = 1
        
        while len(movie_reviews) < num_reviews_per_movie:
            try:
                driver.get(f"{movie_url}/reviews/page/{page}/")
                review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.film-detail-content')
                
                for element in review_elements:
                    if len(movie_reviews) >= num_reviews_per_movie:
                        break
                    
                    review = {}
                    try:
                        # Scrape user review details
                        review_text_element = element.find_element(By.CSS_SELECTOR, 'div.body-text.-prose.collapsible-text')
                        review['review_text'] = review_text_element.text
                    except NoSuchElementException:
                        review['review_text'] = None

                    try:
                        rating_element = element.find_element(By.CSS_SELECTOR, 'span.rating')
                        review['rating'] = normalize_rating(rating_element.text)
                    except NoSuchElementException:
                        review['rating'] = None

                    try:
                        user_url_element = element.find_element(By.CSS_SELECTOR, 'h2.headline-2 a')
                        review['user_url'] = f"{user_url_element.get_attribute('href')}"
                    except NoSuchElementException:
                        review['user_url'] = None
                    
                    try:
                        review_date_element = element.find_element(By.CSS_SELECTOR, 'span.date')
                        review['review_date'] = review_date_element.text
                    except NoSuchElementException:
                        review['review_date'] = None
                    
                    review['movie_url'] = movie_url
                    
                    # Only add the review if it has text and a rating
                    if review['review_text'] and review['rating'] is not None:
                        movie_reviews.append(review)

                # Go to next page if needed
                if len(movie_reviews) < num_reviews_per_movie:
                    page += 1
                else:
                    break
            
            except TimeoutException:
                print(f"Timeout while loading page {page} for movie {movie_url}")
                break
        
        all_reviews.extend(movie_reviews)
    
    df = pd.DataFrame(all_reviews)
    print("Reviews successfully scraped from movie URLs!\nReviews preview:")
    print(df.head().to_string())
    print("\nClosing connection to selenium WebDriver...")
    driver.quit()
    print("Connection closed successfully!\n\n")
    return df

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, TimeoutException

def normalize_rating(star_rating):
    rating_map = {'★': 0.2, '½': 0.1}
    return sum(rating_map[char] for char in star_rating if char in rating_map)


def scrape_reviews_from_movies(movie_urls, num_reviews_per_movie):
    # Setup WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    print("Attempting to connect to Selenium WebDriver...")
    driver = webdriver.Remote(
        command_executor='http://selenium:4444/wd/hub',
        options=options
    )
    print("Successfully connected to WebDriver!")
    
    all_reviews = []

    for movie_url in movie_urls.url:
        print(f"Scraping reviews for movie: {movie_url}")
        movie_reviews = []
        page = 1
        
        while len(movie_reviews) < num_reviews_per_movie:
            try:
                driver.get(f"{movie_url}/reviews/page/{page}/")
                review_elements = driver.find_elements(By.CSS_SELECTOR, 'li.film-detail')
                
                for element in review_elements:
                    if len(movie_reviews) >= num_reviews_per_movie:
                        break
                    
                    review = {}
                    try:
                        # Scrape user URL
                        user_url_element = element.find_element(By.CSS_SELECTOR, 'a.avatar.-a40')
                        review['user_url'] = user_url_element.get_attribute('href')
                    except NoSuchElementException:
                        review['user_url'] = None

                    try:
                        # Scrape review text
                        review_text_element = element.find_element(By.CSS_SELECTOR, 'div.body-text.-prose.collapsible-text')
                        review['review_text'] = review_text_element.text
                    except NoSuchElementException:
                        review['review_text'] = None

                    try:
                        # Scrape rating
                        rating_element = element.find_element(By.CSS_SELECTOR, 'span.rating')
                        review['rating'] = normalize_rating(rating_element.text)
                    except NoSuchElementException:
                        review['rating'] = None

                    try:
                        # Scrape review date
                        review_date_element = element.find_element(By.CSS_SELECTOR, 'span.date > span._nobr')
                        review['review_date'] = review_date_element.text
                    except NoSuchElementException:
                        review['review_date'] = None
                    
                    review['movie_url'] = movie_url
                    
                    # Only add the review if it has text and a rating
                    if review['review_text'] and review['rating'] is not None:
                        movie_reviews.append(review)

                # Go to next page if needed
                if len(movie_reviews) < num_reviews_per_movie:
                    page += 1
                else:
                    break
            
            except TimeoutException:
                print(f"Timeout while loading page {page} for movie {movie_url}")
                break
        
        all_reviews.extend(movie_reviews)
    
    df = pd.DataFrame(all_reviews)
    print("Reviews successfully scraped from movie URLs!\nReviews preview:")
    print(df.head().to_string())
    print("\nClosing connection to selenium WebDriver...")
    driver.quit()
    print("Connection closed successfully!\n\n")
    return df

In [8]:
def scrape_reviews_from_movies(movie_urls, num_reviews_per_movie):
    # Setup WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    print("Attempting to connect to Selenium WebDriver...")
    driver = webdriver.Remote(
        command_executor='http://selenium:4444/wd/hub',
        options=options
    )
    print("Successfully connected to WebDriver!")
    
    all_reviews = []

    for movie_url in movie_urls.url:
        print(f"Scraping reviews for movie: {movie_url}")
        movie_reviews = []
        page = 1
        
        try:
            driver.get(f"{movie_url}/reviews/")
            
            # Scrape movie title and release year
            try:
                movie_title_element = driver.find_element(By.CSS_SELECTOR, 'div.contextual-title h1.headline-2 a')
                movie_title = movie_title_element.text
            except NoSuchElementException:
                movie_title = None
            
            try:
                movie_year_element = driver.find_element(By.CSS_SELECTOR, 'div.contextual-title h1.headline-2 small.metadata a')
                movie_year = movie_year_element.text
            except NoSuchElementException:
                movie_year = None

            while len(movie_reviews) < num_reviews_per_movie:
                review_elements = driver.find_elements(By.CSS_SELECTOR, 'li.film-detail')
                
                for element in review_elements:
                    if len(movie_reviews) >= num_reviews_per_movie:
                        break
                    
                    review = {}
                    review['movie_title'] = movie_title
                    review['movie_year'] = movie_year

                    try:
                        # Scrape user URL
                        user_url_element = element.find_element(By.CSS_SELECTOR, 'a.avatar.-a40')
                        review['user_url'] = user_url_element.get_attribute('href')
                    except NoSuchElementException:
                        review['user_url'] = None

                    try:
                        # Scrape review text
                        review_text_element = element.find_element(By.CSS_SELECTOR, 'div.body-text.-prose.collapsible-text')
                        review['review_text'] = review_text_element.text
                    except NoSuchElementException:
                        review['review_text'] = None

                    try:
                        # Scrape rating
                        rating_element = element.find_element(By.CSS_SELECTOR, 'span.rating')
                        review['rating'] = normalize_rating(rating_element.text)
                    except NoSuchElementException:
                        review['rating'] = None

                    try:
                        # Scrape review date
                        review_date_element = element.find_element(By.CSS_SELECTOR, 'span.date > span._nobr')
                        review['review_date'] = review_date_element.text
                    except NoSuchElementException:
                        review['review_date'] = None
                    
                    review['movie_url'] = movie_url
                    
                    # Only add the review if it has text and a rating
                    if review['review_text'] and review['rating'] is not None:
                        movie_reviews.append(review)

                # Go to next page if needed
                if len(movie_reviews) < num_reviews_per_movie:
                    page += 1
                    driver.get(f"{movie_url}/reviews/page/{page}/")
                else:
                    break
            
        except TimeoutException:
            print(f"Timeout while loading reviews for movie {movie_url}")
        
        all_reviews.extend(movie_reviews)
    
    df = pd.DataFrame(all_reviews)
    print("Reviews successfully scraped from movie URLs!\nReviews preview:")
    print(df.head().to_string())
    print("\nClosing connection to selenium WebDriver...")
    driver.quit()
    print("Connection closed successfully!\n\n")
    return df

In [9]:
movie_reviews = scrape_reviews_from_movies(movie_urls, num_reviews_per_movie=1)
movie_reviews

Attempting to connect to Selenium WebDriver...
Successfully connected to WebDriver!
Scraping reviews for movie: https://letterboxd.com/film/barbie/
Scraping reviews for movie: https://letterboxd.com/film/parasite-2019/
Scraping reviews for movie: https://letterboxd.com/film/everything-everywhere-all-at-once/
Scraping reviews for movie: https://letterboxd.com/film/fight-club/
Scraping reviews for movie: https://letterboxd.com/film/la-la-land/
Reviews successfully scraped from movie URLs!
Reviews preview:
                         movie_title movie_year                            user_url                                                                                                                                                                                                                                                                                                                                    review_text  rating  review_date                                                    

,movie_title,movie_year,user_url,review_text,rating,review_date,movie_url
0,Barbie,2023,https://letterboxd.com/dovade/,"Gesellschaftlich wichtige Themen angesprochen,...",0.6,20 Aug 2024,https://letterboxd.com/film/barbie/
1,Parasite,2019,https://letterboxd.com/tmarg0t3/,this made me so incredibly nervous!\nit was ab...,1.0,20 Aug 2024,https://letterboxd.com/film/parasite-2019/
2,Everything Everywhere All at Once,2022,https://letterboxd.com/_tateisi/,"simplesmente incrível. pqp q filme perfeito, e...",1.0,17 May 2022,https://letterboxd.com/film/everything-everywh...
3,Fight Club,1999,https://letterboxd.com/clem005/,The ending gave me goosebumps. Even After seco...,1.0,20 Aug 2024,https://letterboxd.com/film/fight-club/
4,La La Land,2016,https://letterboxd.com/marchrains/,I don’t know why I rewatched this knowing it w...,1.0,20 Aug 2024,https://letterboxd.com/film/la-la-land/


In [7]:
movie_reviews = scrape_reviews_from_movies(movie_urls, num_reviews_per_movie=1)
movie_reviews

Attempting to connect to Selenium WebDriver...
Successfully connected to WebDriver!
Scraping reviews for movie: https://letterboxd.com/film/barbie/
Scraping reviews for movie: https://letterboxd.com/film/parasite-2019/
Scraping reviews for movie: https://letterboxd.com/film/everything-everywhere-all-at-once/
Scraping reviews for movie: https://letterboxd.com/film/fight-club/
Scraping reviews for movie: https://letterboxd.com/film/la-la-land/
Reviews successfully scraped from movie URLs!
Reviews preview:
                                user_url                                                                                                                                                       review_text  rating  review_date                                                       movie_url
0   https://letterboxd.com/st4rgirlally/                                                                                                                                             Barbie summer forever 

,user_url,review_text,rating,review_date,movie_url
0,https://letterboxd.com/st4rgirlally/,Barbie summer forever,1.0,20 Aug 2024,https://letterboxd.com/film/barbie/
1,https://letterboxd.com/milescampbell/,Yes the hype is real.,1.0,28 Apr 2024,https://letterboxd.com/film/parasite-2019/
2,https://letterboxd.com/planeswalker/,Slow start but hell for me it has literally ev...,1.0,10 Feb 2024,https://letterboxd.com/film/everything-everywh...
3,https://letterboxd.com/jossuush/,This is how it feels when my friends hit me wh...,1.0,20 Aug 2024,https://letterboxd.com/film/fight-club/
4,https://letterboxd.com/cs_leho/,Don’t watch this if you are in a complicated r...,0.8,15 Aug 2024,https://letterboxd.com/film/la-la-land/


In [5]:
movie_reviews = scrape_reviews_from_movies(movie_urls, num_reviews_per_movie=1)
movie_reviews

Attempting to connect to Selenium WebDriver...
Successfully connected to WebDriver!
Scraping reviews for movie: https://letterboxd.com/film/barbie/
Scraping reviews for movie: https://letterboxd.com/film/parasite-2019/
Scraping reviews for movie: https://letterboxd.com/film/everything-everywhere-all-at-once/
Scraping reviews for movie: https://letterboxd.com/film/fight-club/
Scraping reviews for movie: https://letterboxd.com/film/la-la-land/
Reviews successfully scraped from movie URLs!
Reviews preview:
                                                                                                                                                                                                                                                                                                                                                                                                                           review_text rating user_url                                   review_date       

,review_text,rating,user_url,review_date,movie_url
0,Just there is Ryan Gosling,★★★½,None,Watched by Alex G 20 Aug 2024,https://letterboxd.com/film/barbie/
1,"Em Parasita, Bong Joon-ho expõe os abismos que...",★★★★★,None,Rewatched by Rodolfo Zolet Palma 20 Aug 2024,https://letterboxd.com/film/parasite-2019/
2,holy shit man,★★★★★,None,Watched by aleks_bnd 20 Aug 2024,https://letterboxd.com/film/everything-everywh...
3,“Tu m’as rencontré à un moment étrange de mon ...,★★★★★,None,Watched by Toinouuuuu 20 Aug 2024,https://letterboxd.com/film/fight-club/
4,this movie is everything. my new favorite movi...,★★★★★,None,Watched by filmmovieari 20 Aug 2024,https://letterboxd.com/film/la-la-land/


## 2A) Scrape from users

In [1]:
import argparse
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

def scrape_user_urls(num_users):
    # Setup WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    print("Attempting to connect to Selenium WebDriver...")
    driver = webdriver.Remote(
        command_executor='http://selenium:4444/wd/hub',
        options=options
    )
    print("...Connected to selenium service!")
    user_urls = []
    page = 1
    while len(user_urls) < num_users:
        driver.get(f"https://letterboxd.com/members/popular/this/week/page/{page}/")
        elements = driver.find_elements(By.CSS_SELECTOR, 'a.name')
        for element in elements:
            if len(user_urls) >= num_users:
                break
            user_urls.append(element.get_attribute('href'))
        page += 1
    
    df = pd.DataFrame(user_urls, columns=['url'])
    print("User urls sucessfully scraped!\nUser urls preview:")
    print(df.head().to_string())
    print("\nClosing connection to selenium webdriver...")
    driver.quit()
    print("Connection closed sucessfully!\n\n")
    return df


In [2]:
user_urls = scrape_user_urls(10)

Attempting to connect to Selenium WebDriver...
...Connected to selenium service!
User urls sucessfully scraped!
User urls preview:
                                    url
0      https://letterboxd.com/kurstboy/
1  https://letterboxd.com/schaffrillas/
2           https://letterboxd.com/jay/
3    https://letterboxd.com/deathproof/
4     https://letterboxd.com/jaragon23/

Closing connection to selenium webdriver...
Connection closed sucessfully!




## 2b) Scrape reviews from user urls

In [31]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
print("Attempting to connect to Selenium WebDriver...")
driver = webdriver.Remote(
    command_executor='http://selenium:4444/wd/hub',
    options=options
)
print("Successfully connected to WebDriver!")

Attempting to connect to Selenium WebDriver...
Successfully connected to WebDriver!


In [32]:
user_url = user_urls.url[0] ; page=1
driver.get(f"{user_url}/films/reviews/by/added/page/{page}/")
element = driver.find_elements(By.CSS_SELECTOR, 'div.film-detail-content')[0]
review_text_element = element.find_element(By.CSS_SELECTOR, 'div.body-text.-prose.collapsible-text')
review_text = review_text_element.text if review_text_element else None

rating_element = element.find_element(By.CSS_SELECTOR, 'span.rating')
star_rating = rating_element.text if rating_element else None

In [35]:
element.find_element(By.CSS_SELECTOR, "h2.headline-2 a").text

'Labyrinth'

In [17]:
driver.close()

In [ ]:
<a href="/film/alien-resurrection/" class="frame has-menu"><span class="frame-title">Alien Resurrection (1997)</span><span class="overlay"></span><span class="overlay-actions js-film-options -w70" style="display: none;"><span class="film-watch-link-target" data-film-id="47887"><span class="film-watch-link"><span class="has-icon icon-16 icon-watch ajax-click-action" data-action="/film/alien-resurrection/mark-as-watched/"><span class="replace icon"></span>Seen this film?</span></span></span><span class="like-link-target" data-likeable-uid="film:47887"><span class="like-link"><span class="has-icon icon-16 ajax-click-action  icon-like" data-action="/s/film:47887/like/" data-recaptcha-action="film_like" data-confirm="" data-modal-confirm="true"><span class="icon"></span>Like this film?</span></span></span><span class="replace menu-link icon"></span></span></a>

In [3]:
def normalize_rating(star_rating):
    rating_map = {'★': 0.2, '½': 0.1}
    return sum(rating_map[char] for char in star_rating if char in rating_map)

def scrape_reviews_from_users(user_urls, num_reviews_per_user):
    # Setup WebDriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    print("Attempting to connect to Selenium WebDriver...")
    driver = webdriver.Remote(
        command_executor='http://selenium:4444/wd/hub',
        options=options
    )
    print("Successfully connected to WebDriver!")
    
    all_reviews = []

    for user_url in user_urls.url:
        print(f"Scraping reviews for user: {user_url}")
        user_reviews = []
        page = 1
        while len(user_reviews) < num_reviews_per_user:
            driver.get(f"{user_url}/films/reviews/by/added/page/{page}/")
            review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.film-detail-content')
            
            for element in review_elements:
                if len(user_reviews) >= num_reviews_per_user:
                    break
                
                review_text_element = element.find_element(By.CSS_SELECTOR, 'div.body-text.-prose.collapsible-text')
                review_text = review_text_element.text if review_text_element else None
                
                rating_element = element.find_element(By.CSS_SELECTOR, 'span.rating')
                star_rating = rating_element.text if rating_element else None
                
                if review_text and star_rating is not None:
                    user_reviews.append({
                        'user_url': user_url,
                        'review': review_text,
                        'rating': normalize_rating(star_rating)
                    })
            
            # Go to next page if needed
            if len(user_reviews) < num_reviews_per_user:
                page += 1
            else:
                break
        
        all_reviews.extend(user_reviews)
    
    df = pd.DataFrame(all_reviews)
    print("Reviews sucessfully scraped from user urls!\nReviews preview:")
    print(df.head().to_string())
    print("\nClosing connection to selenium webdriver...")
    driver.quit()
    print("Connection closed sucessfully!\n\n")
    return df

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--num_users', type=int, default=10, help='Number of user URLs to scrape')
    parser.add_argument('--num_reviews_per_user', type=int, default=10, help='Number of reviews to scrape per user')
    args = parser.parse_args()
    
    user_urls = scrape_user_urls(args.num_users)
    reviews_from_users = scrape_reviews_from_users(user_urls, args.num_reviews_per_user)
    reviews_from_users.to_csv('/app/data/reviews_from_users.csv', index=False)

## Attempt 2 at scraping user info

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from datetime import datetime

In [4]:
def normalize_rating(star_rating):
    rating_map = {'★': 0.2, '½': 0.1}
    return sum(rating_map[char] for char in star_rating if char in rating_map)


In [18]:
# Setup WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
print("Attempting to connect to Selenium WebDriver...")
driver = webdriver.Remote(
    command_executor='http://selenium:4444/wd/hub',
    options=options
)
print("Successfully connected to WebDriver!")

Attempting to connect to Selenium WebDriver...
Successfully connected to WebDriver!


In [19]:
def scrape_reviews_from_users(user_urls, num_reviews_per_user):
    all_reviews = []

    for user_url in user_urls.url:
        print(f"Scraping reviews for user: {user_url}")
        user_reviews = []
        page = 1
        while len(user_reviews) < num_reviews_per_user:
            try:
                driver.get(f"{user_url}/films/reviews/by/added/page/{page}/")
                review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.film-detail-content')
                
                for element in review_elements:
                    if len(user_reviews) >= num_reviews_per_user:
                        break
                    
                    review = {}
                    try:
                        # Scrape movie details
                        movie_name_element = element.find_element(By.CSS_SELECTOR, 'h2.headline-2 a')
                        review['movie_name'] = movie_name_element.text
                        review['movie_url'] = f"{movie_name_element.get_attribute('href')}"
                    except NoSuchElementException:
                        review['movie_name'] = None
                        review['movie_url'] = None

                    try:
                        # Scrape review
                        review_text_element = element.find_element(By.CSS_SELECTOR, 'div.body-text.-prose.collapsible-text')
                        review['review_text'] = review_text_element.text
                    except NoSuchElementException:
                        review['review_text'] = None

                    try:
                        # Scrape rating
                        rating_element = element.find_element(By.CSS_SELECTOR, 'span.rating')
                        review['rating'] = normalize_rating(rating_element.text)
                    except NoSuchElementException:
                        review['rating'] = None

                    try:
                        # Additional details
                        review_date_element = element.find_element(By.CSS_SELECTOR, 'span.date')
                        review['review_date'] = review_date_element.text
                    except NoSuchElementException:
                        review['review_date'] = None

                    try:
                        movie_release_date_element = element.find_element(By.CSS_SELECTOR, 'small.metadata a')
                        review['movie_release_date'] = movie_release_date_element.text
                    except NoSuchElementException:
                        review['movie_release_date'] = None

                    review['user_url'] = user_url

                    if review['review_text'] and review['rating'] is not None:
                        user_reviews.append(review)

                # Go to next page if needed
                if len(user_reviews) < num_reviews_per_user:
                    page += 1
                else:
                    break
            
            except TimeoutException:
                print(f"Timeout while loading page {page} for user {user_url}")
                break
        
        all_reviews.extend(user_reviews)
    
    df = pd.DataFrame(all_reviews)
    print("Reviews successfully scraped from user URLs!\nReviews preview:")
    print(df.head().to_string())
    print("\nClosing connection to selenium WebDriver...")
    driver.quit()
    print("Connection closed successfully!\n\n")
    return df

In [20]:
user_reviews = scrape_reviews_from_users(user_urls, num_reviews_per_user=1)

Scraping reviews for user: https://letterboxd.com/kurstboy/
Scraping reviews for user: https://letterboxd.com/schaffrillas/
Scraping reviews for user: https://letterboxd.com/jay/
Scraping reviews for user: https://letterboxd.com/deathproof/
Scraping reviews for user: https://letterboxd.com/jaragon23/
Scraping reviews for user: https://letterboxd.com/colonelmortimer/
Scraping reviews for user: https://letterboxd.com/jeaba/
Scraping reviews for user: https://letterboxd.com/24framesofnick/
Scraping reviews for user: https://letterboxd.com/thejoshl/
Scraping reviews for user: https://letterboxd.com/ingridgoeswest/
Reviews successfully scraped from user URLs!
Reviews preview:
                       movie_name                                                             movie_url                                                                                                                                                                                                                         

In [21]:
user_reviews

,movie_name,movie_url,review_text,rating,review_date,movie_release_date,user_url
0,Bobo the Monkey,https://letterboxd.com/kurstboy/film/bobo-the-...,monkey mondays #33,0.8,Watched 19 Aug 2024,2021,https://letterboxd.com/kurstboy/
1,Alien Resurrection,https://letterboxd.com/schaffrillas/film/alien...,One of the dumbest franchise sequels I've ever...,0.5,Watched 18 Aug 2024,1997,https://letterboxd.com/schaffrillas/
2,Alien: Romulus,https://letterboxd.com/jay/film/alien-romulus/,they grow up so fast,0.5,Watched 19 Aug 2024,2024,https://letterboxd.com/jay/
3,The Idea of You,https://letterboxd.com/deathproof/film/the-ide...,near the end when anne hathaway is flipping th...,0.4,Watched 10 May 2024,2024,https://letterboxd.com/deathproof/
4,SPL 2: A Time for Consequences,https://letterboxd.com/jaragon23/film/spl-2-a-...,Mesmerizing. Could have watched Tony Jaa and W...,0.8,Watched 16 Aug 2024,2015,https://letterboxd.com/jaragon23/
5,Alien: Romulus,https://letterboxd.com/colonelmortimer/film/al...,The parts that are an ALIEN splatter movie are...,0.6,Watched 12 Aug 2024,2024,https://letterboxd.com/colonelmortimer/
6,Alien: Romulus,https://letterboxd.com/jeaba/film/alien-romulus/,i guess mining ain’t lookin so bad anymore,0.8,Watched 19 Aug 2024,2024,https://letterboxd.com/jeaba/
7,Alien: Covenant,https://letterboxd.com/24framesofnick/film/ali...,I was smoking crack on my first watch\nHoly fu...,0.8,Rewatched 16 Aug 2024,2017,https://letterboxd.com/24framesofnick/
8,Barry Lyndon,https://letterboxd.com/thejoshl/film/barry-lyn...,Having already proven himself a visionary of w...,1.0,Rewatched 18 Aug 2024,1975,https://letterboxd.com/thejoshl/
9,Longlegs,https://letterboxd.com/ingridgoeswest/film/lon...,Maika monroe you are irreplaceable,0.7,Watched 12 Jul 2024,2024,https://letterboxd.com/ingridgoeswest/


In [18]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
print("Attempting to connect to Selenium WebDriver...")
driver = webdriver.Remote(
    command_executor='http://selenium:4444/wd/hub',
    options=options
)
print("Successfully connected to WebDriver!")

Attempting to connect to Selenium WebDriver...
Successfully connected to WebDriver!


In [28]:
num_reviews_per_user=1
user_url = user_urls.url[0] 
user_url

'https://letterboxd.com/kurstboy/'

In [20]:
driver.get(f"{user_url}/films/reviews/by/added/page/{page}/")

In [21]:
review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.film-detail-content')

In [24]:
element = review_elements[0]
review = {}

In [26]:
try:
    # Scrape movie details
    movie_name_element = element.find_element(By.CSS_SELECTOR, 'h2.headline-2 a')
    review['movie_name'] = movie_name_element.text
    review['movie_url'] = f"https://letterboxd.com{movie_name_element.get_attribute('href')}"
except NoSuchElementException:
    review['movie_name'] = None
    review['movie_url'] = None

In [29]:
try:
    review_text_element = element.find_element(By.CSS_SELECTOR, 'div.body-text.-prose.collapsible-text')
    review['review_text'] = review_text_element.text
except NoSuchElementException:
    review['review_text'] = None

try:
    rating_element = element.find_element(By.CSS_SELECTOR, 'span.rating')
    review['rating'] = normalize_rating(rating_element.text)
except NoSuchElementException:
    review['rating'] = None

try:
    # Additional details
    review_date_element = element.find_element(By.CSS_SELECTOR, 'span.date')
    review['review_date'] = review_date_element.text
except NoSuchElementException:
    review['review_date'] = None

try:
    movie_release_date_element = element.find_element(By.CSS_SELECTOR, 'small.metadata a')
    review['movie_release_date'] = movie_release_date_element.text
except NoSuchElementException:
    review['movie_release_date'] = None

try:
    genre_element = element.find_element(By.CSS_SELECTOR, 'span.genre')
    review['genre'] = genre_element.text
except NoSuchElementException:
    review['genre'] = None


review['user_url'] = user_url

In [30]:
review

{'movie_name': 'Bobo the Monkey',
 'movie_url': 'https://letterboxd.comhttps://letterboxd.com/kurstboy/film/bobo-the-monkey/',
 'review_text': 'monkey mondays #33',
 'rating': 0.8,
 'review_date': 'Watched 19 Aug 2024',
 'movie_release_date': '2021',
 'genre': None,
 'short_description': 'monkey mondays #33',
 'user_url': 'https://letterboxd.com/kurstboy/'}

In [31]:
driver.close()

InvalidSessionIdException: Message: Unable to find session with ID: e42cdd0ad2cfccdab6f204ca4cbd06a7
Build info: version: '4.23.0', revision: '77010cd'
System info: os.name: 'Linux', os.arch: 'amd64', os.version: '6.8.0-40-generic', java.version: '17.0.11'
Driver info: driver.version: unknown
Stacktrace:
org.openqa.selenium.NoSuchSessionException: Unable to find session with ID: e42cdd0ad2cfccdab6f204ca4cbd06a7
Build info: version: '4.23.0', revision: '77010cd'
System info: os.name: 'Linux', os.arch: 'amd64', os.version: '6.8.0-40-generic', java.version: '17.0.11'
Driver info: driver.version: unknown
	at org.openqa.selenium.grid.sessionmap.local.LocalSessionMap.get(LocalSessionMap.java:132)
	at org.openqa.selenium.grid.sessionmap.SessionMap.getUri(SessionMap.java:84)
	at org.openqa.selenium.grid.router.HandleSession.lambda$loadSessionId$4(HandleSession.java:223)
	at io.opentelemetry.context.Context.lambda$wrap$2(Context.java:224)
	at org.openqa.selenium.grid.router.HandleSession.execut

In [10]:
all_reviews = []

for user_url in user_urls.url:
    print(f"Scraping reviews for user: {user_url}")
    user_reviews = []
    page = 1
    try:
        driver.get(f"{user_url}/films/reviews/by/added/page/{page}/")
        review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.film-detail-content')
        
        for element in review_elements:
            if len(user_reviews) >= num_reviews_per_user:
                break
            
            review = {}
            try:
                # Scrape movie details
                movie_name_element = element.find_element(By.CSS_SELECTOR, 'h2.headline-2 a')
                review['movie_name'] = movie_name_element.text
                review['movie_url'] = f"https://letterboxd.com{movie_name_element.get_attribute('href')}"
            except NoSuchElementException:
                review['movie_name'] = None
                review['movie_url'] = None
    except TimeoutException:
                print(f"Timeout while loading page {page} for user {user_url}")
                break

Scraping reviews for user: https://letterboxd.com/kurstboy/


InvalidSessionIdException: Message: Unable to find session with ID: 0c3050a3278c324edb501691659d1637
Build info: version: '4.23.0', revision: '77010cd'
System info: os.name: 'Linux', os.arch: 'amd64', os.version: '6.8.0-40-generic', java.version: '17.0.11'
Driver info: driver.version: unknown
Stacktrace:
org.openqa.selenium.NoSuchSessionException: Unable to find session with ID: 0c3050a3278c324edb501691659d1637
Build info: version: '4.23.0', revision: '77010cd'
System info: os.name: 'Linux', os.arch: 'amd64', os.version: '6.8.0-40-generic', java.version: '17.0.11'
Driver info: driver.version: unknown
	at org.openqa.selenium.grid.sessionmap.local.LocalSessionMap.get(LocalSessionMap.java:132)
	at org.openqa.selenium.grid.sessionmap.SessionMap.getUri(SessionMap.java:84)
	at org.openqa.selenium.grid.router.HandleSession.lambda$loadSessionId$4(HandleSession.java:223)
	at io.opentelemetry.context.Context.lambda$wrap$2(Context.java:224)
	at org.openqa.selenium.grid.router.HandleSession.execut

In [12]:
try:
    # Scrape movie details
    movie_name_element = element.find_element(By.CSS_SELECTOR, 'h2.headline-2 a')
    review['movie_name'] = movie_name_element.text
    review['movie_url'] = f"https://letterboxd.com{movie_name_element.get_attribute('href')}"
except NoSuchElementException:
    review['movie_name'] = None
    review['movie_url'] = None

NameError: name 'element' is not defined

In [13]:
review_elements

NameError: name 'review_elements' is not defined